In [1]:
import numpy as np
import pandas as pd
import os
import datetime
import time
import matplotlib.pyplot as plt
import pandavro as pdx
from shl.prepare import fill_na, calculate_abs_values, calculate_change, calculate_window, calculate_shift

os.chdir('../')

pd.set_option('display.max_columns', None)

In [2]:
def prepare_features(df, abs_columns, window_sizes, window_center, window_functions, shift_periods, shift_columns_pattern):  
        
    # Add time rounded to seconds
    df['epoch_time'] = df['epoch_time'].round(-3)
    
    # Group values by rounded time
    df = df.groupby(['epoch_time'], as_index=False).mean().drop(['Ignore1', 'Ignore2'], axis=1)
    
    # Ensure that values are sorted before calculating diff
    df = df.sort_values(by='epoch_time')    
    
    # Calculate difference to get distance and speed
    df_diff = calculate_change(df)
    
    # Additional features
    df_diff['distance'] = (df_diff['Latitude_change'].pow(2) + df_diff['Longitude_change'].pow(2)).pow(0.5)
    df_diff['speed'] = df_diff['distance']/df_diff['epoch_time_change']
    df_diff['vertical_speed'] = df_diff['Altitude_change']/df_diff['epoch_time_change']
    df_diff['direction'] = df_diff['Longitude_change']/df_diff['Latitude_change']
    df_diff['vertical_direction'] = df_diff['Altitude_change']/df_diff['distance']
    
    df_diff.drop('epoch_time_change', axis=1, inplace=True) 
    fill_na(df_diff)
    
    # Second diff to get change of the new features
    df_diff_2 = calculate_change(df_diff[['speed', 'vertical_speed', 'direction', 'vertical_direction']])               
    
    # Merge new features back to the main dataframe
    df = df.merge(df_diff, left_index=True, right_index=True, how='left') \
           .merge(df_diff_2, left_index=True, right_index=True, how='left')    
    
    # Additional abs values features
    calculate_abs_values(df, columns=abs_columns)
          
    # Add moving windows features
    df = calculate_window(df, columns=None, functions=window_functions, window_sizes=window_sizes, window_center=True)       
    
    # Add shifted features
    calculate_shift(df, periods=shift_periods, columns_pattern=shift_columns_pattern) 
               
    return df

In [3]:

settings = {
'abs_columns': ['speed_change', 'vertical_speed_change', 'direction_change', 'vertical_direction_change'],
'window_sizes': [5, 10, 30, 60],
'window_center': True,
'window_functions': ['mean', 'std'],
'shift_periods': [-90, 90],
'shift_columns_pattern': 'window_60',
}


input_files = ['./data/validate/Location.parquet', './data/train/Location.parquet', './data/test/Location.parquet']
output_files_templates = ['./data/validate/features_location.', './data/train/features_location.', './data/test/features_location.']

# input_files = ['./data/validate/Location.parquet']
# output_files_templates = ['./data/validate/features_location.']


output_format = 'parquet'
output_files = [template + output_format for template in output_files_templates]

for file_in, file_out in zip(input_files, output_files):
    df = pd.read_parquet(file_in)
    df = prepare_features(df, **settings)
    if output_format == 'parquet':
        df.to_parquet(file_out, index=False)
    elif output_format == 'avro':
        pdx.to_avro(file_out, df)
        


In [4]:
df

,epoch_time,accuracy,Latitude,Longitude,Altitude,accuracy_change,Latitude_change,Longitude_change,Altitude_change,distance,speed,vertical_speed,direction,vertical_direction,speed_change,vertical_speed_change,direction_change,vertical_direction_change,abs_speed_change,abs_vertical_speed_change,abs_direction_change,abs_vertical_direction_change,accuracy_window_5_mean,accuracy_window_5_std,Latitude_window_5_mean,Latitude_window_5_std,Longitude_window_5_mean,Longitude_window_5_std,Altitude_window_5_mean,Altitude_window_5_std,accuracy_change_window_5_mean,accuracy_change_window_5_std,Latitude_change_window_5_mean,Latitude_change_window_5_std,Longitude_change_window_5_mean,Longitude_change_window_5_std,Altitude_change_window_5_mean,Altitude_change_window_5_std,distance_window_5_mean,distance_window_5_std,speed_window_5_mean,speed_window_5_std,vertical_speed_window_5_mean,vertical_speed_window_5_std,direction_window_5_mean,direction_window_5_std,vertical_direction_window_5_mean,vertical_direction_window_5_std,speed_change_window_5_mean,speed_change_window_5_std,vertical_speed_change_window_5_mean,vertical_speed_change_window_5_std,direction_change_window_5_mean,direction_change_window_5_std,vertical_direction_change_window_5_mean,vertical_direction_change_window_5_std,abs_speed_change_window_5_mean,abs_speed_change_window_5_std,abs_vertical_speed_change_window_5_mean,abs_vertical_speed_change_window_5_std,abs_direction_change_window_5_mean,abs_direction_change_window_5_std,abs_vertical_direction_change_window_5_mean,abs_vertical_direction_change_window_5_std,accuracy_window_10_mean,accuracy_window_10_std,Latitude_window_10_mean,Latitude_window_10_std,Longitude_window_10_mean,Longitude_window_10_std,Altitude_window_10_mean,Altitude_window_10_std,accuracy_change_window_10_mean,accuracy_change_window_10_std,Latitude_change_window_10_mean,Latitude_change_window_10_std,Longitude_change_window_10_mean,Longitude_change_window_10_std,Altitude_change_window_10_mean,Altitude_change_window_10_std,distance_window_10_mean,distance_window_10_std,speed_window_10_mean,speed_window_10_std,vertical_speed_window_10_mean,vertical_speed_window_10_std,direction_window_10_mean,direction_window_10_std,vertical_direction_window_10_mean,vertical_direction_window_10_std,speed_change_window_10_mean,speed_change_window_10_std,vertical_speed_change_window_10_mean,vertical_speed_change_window_10_std,direction_change_window_10_mean,direction_change_window_10_std,vertical_direction_change_window_10_mean,vertical_direction_change_window_10_std,abs_speed_change_window_10_mean,abs_speed_change_window_10_std,abs_vertical_speed_change_window_10_mean,abs_vertical_speed_change_window_10_std,abs_direction_change_window_10_mean,abs_direction_change_window_10_std,abs_vertical_direction_change_window_10_mean,abs_vertical_direction_change_window_10_std,accuracy_window_30_mean,accuracy_window_30_std,Latitude_window_30_mean,Latitude_window_30_std,Longitude_window_30_mean,Longitude_window_30_std,Altitude_window_30_mean,Altitude_window_30_std,accuracy_change_window_30_mean,accuracy_change_window_30_std,Latitude_change_window_30_mean,Latitude_change_window_30_std,Longitude_change_window_30_mean,Longitude_change_window_30_std,Altitude_change_window_30_mean,Altitude_change_window_30_std,distance_window_30_mean,distance_window_30_std,speed_window_30_mean,speed_window_30_std,vertical_speed_window_30_mean,vertical_speed_window_30_std,direction_window_30_mean,direction_window_30_std,vertical_direction_window_30_mean,vertical_direction_window_30_std,speed_change_window_30_mean,speed_change_window_30_std,vertical_speed_change_window_30_mean,vertical_speed_change_window_30_std,direction_change_window_30_mean,direction_change_window_30_std,vertical_direction_change_window_30_mean,vertical_direction_change_window_30_std,abs_speed_change_window_30_mean,abs_speed_change_window_30_std,abs_vertical_speed_change_window_30_mean,abs_vertical_speed_change_window_30_std,abs_direction_change_window_30_m

### Export for analysis

In [32]:
# Join labels for analysis
# features_file = './data/validate/features_location.parquet'
# features = pd.read_parquet(features_file)

# label_file = './data/validate/Label.parquet'
# label = pd.read_parquet(label_file)

# features = label.merge(features, how='outer').sort_values(by='epoch_time')
# features.to_csv('./data/location_analysis.csv')
# features

,epoch_time,label,accuracy,Latitude,Longitude,Altitude,accuracy_change,Latitude_change,Longitude_change,Altitude_change,distance,speed,vertical_speed,direction,vertical_direction,speed_change,vertical_speed_change,direction_change,vertical_direction_change,abs_speed_change,abs_vertical_speed_change,abs_direction_change,abs_vertical_direction_change,accuracy_3_s_window_avg,Latitude_3_s_window_avg,Longitude_3_s_window_avg,Altitude_3_s_window_avg,accuracy_change_3_s_window_avg,Latitude_change_3_s_window_avg,Longitude_change_3_s_window_avg,Altitude_change_3_s_window_avg,distance_3_s_window_avg,speed_3_s_window_avg,vertical_speed_3_s_window_avg,direction_3_s_window_avg,vertical_direction_3_s_window_avg,speed_change_3_s_window_avg,vertical_speed_change_3_s_window_avg,direction_change_3_s_window_avg,vertical_direction_change_3_s_window_avg,abs_speed_change_3_s_window_avg,abs_vertical_speed_change_3_s_window_avg,abs_direction_change_3_s_window_avg,abs_vertical_direction_change_3_s_window_avg,speed_10_s_window_avg,speed_30_s_window_avg,speed_60_s_window_avg,speed_60_s_dev_from_mean_squared,speed_60_s_std
143958,1497426504000,NaN,96.0,50.826394,-0.119515,248.937172,32.0,3.804330e-07,-1.255605e-05,-119.574615,1.256181e-05,6.280907e-09,-0.059787,-33.004632,-9.518897e+06,1.680762e-08,0.060520,8.351667e+01,9.550650e+06,1.680762e-08,0.060520,8.351667e+01,9.550650e+06,106.666667,50.826394,-0.119531,169.465137,10.666667,6.884777e-07,-2.451935e-05,-39.584234,2.454293e-05,2.244929e-08,-0.019655,-0.064915,-3.161602e+06,-2.093636e-09,0.019929,-4.370930,-19.024247,2.322533e-08,2.041785e-02,19.671966,6.554598e+01,4.061749e-08,3.942080e-08,2.986638e-08,3.809006e-15,1.735828e-16
143959,1497426506000,NaN,128.0,50.826394,-0.119528,129.362557,32.0,3.804330e-07,-1.255605e-05,-119.574615,1.256181e-05,6.280907e-09,-0.059787,-33.004632,-9.518897e+06,1.680762e-08,0.060520,8.351667e+01,9.550650e+06,1.680762e-08,0.060520,8.351667e+01,9.550650e+06,106.666667,50.826394,-0.119531,169.465137,10.666667,6.884777e-07,-2.451935e-05,-39.584234,2.454293e-05,2.244929e-08,-0.019655,-0.064915,-3.161602e+06,-2.093636e-09,0.019929,-4.370930,-19.024247,2.322533e-08,2.041785e-02,19.671966,6.554598e+01,4.061749e-08,3.942080e-08,2.986638e-08,3.809006e-15,1.735828e-16
143960,1497426507000,NaN,96.0,50.826394,-0.119551,130.095681,-32.0,-4.570000e-07,-2.308400e-05,0.733124,2.308852e-05,2.308852e-08,0.000733,50.512035,3.175276e+04,1.680762e-08,0.060520,8.351667e+01,9.550650e+06,1.680762e-08,0.060520,8.351667e+01,9.550650e+06,117.333333,50.826395,-0.119556,129.880903,10.666667,6.884777e-07,-2.451935e-05,-39.584234,2.454293e-05,2.244929e-08,-0.019655,-0.064915,-3.161602e+06,-2.093636e-09,0.019929,-4.370930,-19.024247,2.322533e-08,2.041785e-02,19.671966,6.554598e+01,4.061749e-08,3.942080e-08,2.986638e-08,3.809006e-15,1.735828e-16
143961,1497426508000,NaN,128.0,50.826396,-0.119589,130.184470,32.0,2.142000e-06,-3.791800e-05,0.088789,3.797845e-05,3.797845e-08,0.000089,-17.702148,2.337889e+03,1.488993e-08,-0.000644,-6.821418e+01,-2.941487e+04,1.488993e-08,0.000644,6.821418e+01,2.941487e+04,117.333333,50.826395,-0.119576,130.154874,0.000000,5.616667e-07,-2.033400e-05,0.273971,2.035566e-05,2.035566e-08,0.000274,-0.064915,-3.161602e+06,-2.093636e-09,0.019929,-4.370930,-19.024247,2.322533e-08,2.041785e-02,19.671966,6.554598e+01,4.061749e-08,3.942080e-08,2.986638e-08,3.809006e-15,1.735828e-16
143962,1497426509000,NaN,128.0,50.826396,-0.119589,130.184470,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,-17.702148,2.337889e+03,-3.797845e-08,-0.000089,-6.821418e+01,-2.941487e+04,3.797845e-08,0.000089,6.821418e+01,2.941487e+04,128.000000,50.826397,-0.119628,130.194329,10.666667,1.856333e-06,-5.176900e-05,0.039455,5.180582e-05,5.180582e-08,0.000039,-0.064915,-3.161602e+06,3.145016e-08,-0.000235,-4.370930,-19.024247,5.676913e-08,2.542336e-04,19.671966,6.554598e+01,4.061749e-08,3.942080e-08,2.986638e-08,3.809006e-15,1.735828e-16
...,...,...,...,...,...,...,...,...,...,...,...,..

In [2]:
input_files = './data/validate/Location.parquet'
df = pd.read_parquet(input_files)
df_diff = calculate_change(df)

NameError: name 'fill_na' is not defined

In [ ]:
df_diff